In [7]:
#dependencies 
#had to 
import pandas as pd 
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import config 
from coincallandwritepythonfile import refined_coin_list

In [8]:
#still have issue with a blank line showing 
refined_coin_list

,id,name,symbol,rank,File_Name,injest_datetime
1,1,Bitcoin,BTC,1,Coin_Ranking_Dim,04/22/2022 10:04:00
2,1027,Ethereum,ETH,2,Coin_Ranking_Dim,04/22/2022 10:04:00
3,825,Tether,USDT,3,Coin_Ranking_Dim,04/22/2022 10:04:00
4,1839,BNB,BNB,4,Coin_Ranking_Dim,04/22/2022 10:04:00
5,3408,USD Coin,USDC,5,Coin_Ranking_Dim,04/22/2022 10:04:00
...,...,...,...,...,...,...
97,1896,0x,ZRX,97,Coin_Ranking_Dim,04/22/2022 10:04:00
98,7455,Audius,AUDIO,98,Coin_Ranking_Dim,04/22/2022 10:04:00
99,2099,ICON,ICX,99,Coin_Ranking_Dim,04/22/2022 10:04:00
100,3801,BORA,BORA,100,Coin_Ranking_Dim,04/22/2022 10:04:00


In [9]:
#get ids for each coin and convert to list
coin_ids = refined_coin_list.id.to_list()
type(coin_ids)
#getting 5 first ids to try in practice run
five_ids_to_try = coin_ids[:5]

In [16]:
def get_coin_info_reorder(list_of_coins_, api_key):
    df = pd.DataFrame()
    for id in list_of_coins_:
        url = 'https://pro-api.coinmarketcap.com/v2/cryptocurrency/quotes/latest'
        parameters = {
            'id': id
             }
        headers = {
        'Accepts': 'application/json',
        'X-CMC_PRO_API_KEY': api_key
        }

        session = Session()
        session.headers.update(headers)
        response = session.get(url, params=parameters)
        data = json.loads(response.text)
        #navigate json
        #read to DF to see what we have
        cmc_df = pd.DataFrame.from_dict(data['data'], orient='index')
        df = df.append(cmc_df)
        #need to explode this, then re join 
        #expand nest columns
        #expanded_df = pd.DataFrame.from_dict(cmc_df['quote'][0], orient= 'index')
        #showing USD as an index column, going to drop
        
        #expanded_df.reset_index(drop = True, inplace= True)
        #expanded_df.set_index(pd.Index([count]),inplace= True)
               
        #trial = cmc_df.drop(['quote','tags'], axis= 1)
        #had to reset index or set it manually to match the expanded DF 
    
        #trial.reset_index(drop = True, inplace= True)
        #trial.reset_index(drop = True, inplace= True)
        #trial.set_index(pd.Index([count]),inplace= True)
        
        #had to add key to get them to append properly
        #coin_price_df = pd.concat([trial,expanded_df], axis=1, keys=['trial', 'expanded_df']  )
        #coin_price_df.reset_index(drop = True, inplace= True)

        #this saves it as a coreframe

        #going to try to append this to another dataframe
        #df = pd.DataFrame()
        #coin_price_df.set_index(pd.Index([count]),inplace= True)

        #df = df.append(coin_price_df)
        #df.set_index(pd.Index([count]),inplace= True)
    #had to drop level inorder to get ride of multilevel index   
    # #https://sparkbyexamples.com/pandas/pandas-multiindex-dataframe-examples/?msclkid=fba45bb6c10e11eca1d376f5b274bab7 
    #df = df.droplevel(0, axis =1)
    return df

        


In [18]:
df = get_coin_info_reorder(five_ids_to_try, config.coin_api_key)

In [52]:
#while loop to expand out all of the columns 
i = 0
concat_df = pd.DataFrame()
while i <= 10:
    i+=1
    tags = df['tags'].apply(pd.Series)
    exploded_df = tags[i].apply(pd.Series)
concat_df.append(exploded_df)

,0,category,name,slug
1,NaN,PROPERTY,CMS Holdings Portfolio,cms-holdings-portfolio
1027,NaN,PROPERTY,DCG Portfolio,dcg-portfolio
825,NaN,NaN,NaN,NaN
1839,NaN,NaN,NaN,NaN
3408,NaN,NaN,NaN,NaN


In [71]:
i = 0
number_of_columns = []
while i < len(tags.columns):
    number_of_columns.append(i)
    i+= 1

In [84]:
#while loop to expand out all of the columns 
i = 0
concat_df = pd.DataFrame()
number_of_columns = []
tags = df['tags'].apply(pd.Series)
while i < len(tags.columns):
    number_of_columns.append(i)
    i+=1


In [99]:
tags

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
1,"{'slug': 'mineable', 'name': 'Mineable', 'cate...","{'slug': 'pow', 'name': 'PoW', 'category': 'CO...","{'slug': 'sha-256', 'name': 'SHA-256', 'catego...","{'slug': 'store-of-value', 'name': 'Store Of V...","{'slug': 'state-channel', 'name': 'State Chann...","{'slug': 'coinbase-ventures-portfolio', 'name'...","{'slug': 'three-arrows-capital-portfolio', 'na...","{'slug': 'polychain-capital-portfolio', 'name'...","{'slug': 'binance-labs-portfolio', 'name': 'Bi...","{'slug': 'blockchain-capital-portfolio', 'name...",...,"{'slug': 'a16z-portfolio', 'name': 'a16z Portf...","{'slug': '1confirmation-portfolio', 'name': '1...","{'slug': 'winklevoss-capital-portfolio', 'name...","{'slug': 'usv-portfolio', 'name': 'USV Portfol...","{'slug': 'placeholder-ventures-portfolio', 'na...","{'slug': 'pantera-capital-portfolio', 'name': ...","{'slug': 'multicoin-capital-portfolio', 'name'...","{'slug': 'paradigm-portfolio', 'name': 'Paradi...",NaN,NaN
1027,"{'slug': 'mineable', 'name': 'Mineable', 'cate...","{'slug': 'pow', 'name': 'PoW', 'category': 'CO...","{'slug': 'smart-contracts', 'name': 'Smart Con...","{'slug': 'ethereum-ecosystem', 'name': 'Ethere...","{'slug': 'coinbase-ventures-portfolio', 'name'...","{'slug': 'three-arrows-capital-portfolio', 'na...","{'slug': 'polychain-capital-portfolio', 'name'...","{'slug': 'binance-labs-portfolio', 'name': 'Bi...","{'slug': 'blockchain-capital-portfolio', 'name...","{'slug': 'boostvc-portfolio', 'name': 'BoostVC...",...,"{'slug': 'a16z-portfolio', 'name': 'a16z Portf...","{'slug': '1confirmation-portfolio', 'name': '1...","{'slug': 'winklevoss-capital-portfolio', 'name...","{'slug': 'usv-portfolio', 'name': 'USV Portfol...","{'slug': 'placeholder-ventures-portfolio', 'na...","{'slug': 'pantera-capital-portfolio', 'name': ...","{'slug': 'multicoin-capital-portfolio', 'name'...","{'slug': 'paradigm-portfolio', 'name': 'Paradi...","{'slug': 'injective-ecosystem', 'name': 'Injec...","{'slug': 'bnb-chain', 'name': 'BNB Chain', 'ca..."
825,"{'slug': 'payments', 'name': 'Payments', 'cate...","{'slug': 'stablecoin', 'name': 'Stablecoin', '...","{'slug': 'asset-backed-stablecoin', 'name': 'A...","{'slug': 'avalanche-ecosystem', 'name': 'Avala...","{'slug': 'solana-ecosystem', 'name': 'Solana E...","{'slug': 'arbitrum-ecosytem', 'name': 'Arbitru...","{'slug': 'moonriver-ecosystem', 'name': 'Moonr...","{'slug': 'injective-ecosystem', 'name': 'Injec...","{'slug': 'bnb-chain', 'name': 'BNB Chain', 'ca...","{'slug': 'usd-stablecoin', 'name': 'USD Stable...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1839,"{'slug': 'marketplace', 'name': 'Marketplace',...","{'slug': 'centralized-exchange', 'name': 'Cent...","{'slug': 'payments', 'name': 'Payments', 'cate...","{'slug': 'smart-contracts', 'name': 'Smart Con...","{'slug': 'alameda-research-portfolio', 'name':...","{'slug': 'multicoin-capital-portfolio', 'name'...","{'slug': 'moonriver-ecosystem', 'name': 'Moonr...","{'slug': 'bnb-chain', 'name': 'BNB Chain', 'ca...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3408,"{'slug': 'medium-of-exchange', 'name': 'Medium...","{'slug': 'stablecoin', 'name': 'Stablecoin', '...","{'slug': 'asset-backed-stablecoin', 'name': 'A...","{'slug': 'fantom-ecosystem', 'name': 'Fantom E...","{'slug': 'arbitrum-ecosytem', 'name': 'Arbitru...","{'slug': 'moonriver-ecosystem', 'name': 'Moonr...","{'slug': 'bnb-chain', 'name': 'BNB Chain', 'ca...","{'slug': 'usd-stablecoin', 'name': 'USD Stable...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [97]:
#returns a lot of columns, will need to rename as well
concat_df = pd.DataFrame()
for column in number_of_columns:
    exploded_df = tags[column].apply(pd.Series)   
    concat_df = pd.concat([exploded_df, concat_df], axis=1)    

In [98]:
concat_df

,0,category,name,slug,0,category,name,slug,0,category,...,category,slug,name,category,slug,name,category,slug,name,category
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PROPERTY,...,PROPERTY,sha-256,SHA-256,CONSENSUS_ALGORITHM,pow,PoW,CONSENSUS_ALGORITHM,mineable,Mineable,OTHER
1027,NaN,PROPERTY,BNB Chain,bnb-chain,NaN,PROPERTY,Injective Ecosystem,injective-ecosystem,NaN,PROPERTY,...,PLATFORM,smart-contracts,Smart Contracts,PROPERTY,pow,PoW,CONSENSUS_ALGORITHM,mineable,Mineable,OTHER
825,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,PROPERTY,asset-backed-stablecoin,Asset-Backed Stablecoin,PROPERTY,stablecoin,Stablecoin,PROPERTY,payments,Payments,PROPERTY
1839,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,PROPERTY,payments,Payments,PROPERTY,centralized-exchange,Centralized Exchange,PROPERTY,marketplace,Marketplace,PROPERTY
3408,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,PROPERTY,asset-backed-stablecoin,Asset-Backed Stablecoin,PROPERTY,stablecoin,Stablecoin,PROPERTY,medium-of-exchange,Medium of Exchange,PROPERTY


In [33]:
tags = df['tags'].apply(pd.Series)
tags.columns
tags[0].apply(pd.Series)

,slug,name,category
1,mineable,Mineable,OTHER
1027,mineable,Mineable,OTHER
825,payments,Payments,PROPERTY
1839,marketplace,Marketplace,PROPERTY
3408,medium-of-exchange,Medium of Exchange,PROPERTY


In [24]:
tags = df['tags'].apply(pd.Series)
tags[0]
trial_cleaning = pd.concat([df.drop(['tags'], axis= 1),tags[0]], axis= 1 )
trial_cleaning

,id,name,symbol,slug,num_market_pairs,date_added,max_supply,circulating_supply,total_supply,is_active,platform,cmc_rank,is_fiat,self_reported_circulating_supply,self_reported_market_cap,last_updated,quote,0
1,1,Bitcoin,BTC,bitcoin,9355,2013-04-28T00:00:00.000Z,21000000,1.901898e+07,1.901898e+07,1,None,1,0,None,None,2022-04-22T17:18:00.000Z,"{'USD': {'price': 39523.796835383204, 'volume_...","{'slug': 'mineable', 'name': 'Mineable', 'cate..."
1027,1027,Ethereum,ETH,ethereum,5659,2015-08-07T00:00:00.000Z,None,1.204841e+08,1.204841e+08,1,None,2,0,None,None,2022-04-22T17:17:00.000Z,"{'USD': {'price': 2958.5534929425485, 'volume_...","{'slug': 'mineable', 'name': 'Mineable', 'cate..."
825,825,Tether,USDT,tether,32216,2015-02-25T00:00:00.000Z,None,8.306920e+10,8.561395e+10,1,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ET...",3,0,None,None,2022-04-22T17:17:00.000Z,"{'USD': {'price': 1.0002524796835452, 'volume_...","{'slug': 'payments', 'name': 'Payments', 'cate..."
1839,1839,BNB,BNB,bnb,812,2017-07-25T00:00:00.000Z,165116760,1.632770e+08,1.632770e+08,1,None,4,0,None,None,2022-04-22T17:17:00.000Z,"{'USD': {'price': 407.8732987541661, 'volume_2...","{'slug': 'marketplace', 'name': 'Marketplace',..."
3408,3408,USD Coin,USDC,usd-coin,3790,2018-10-08T00:00:00.000Z,None,4.989235e+10,4.989235e+10,1,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ET...",5,0,None,None,2022-04-22T17:17:00.000Z,"{'USD': {'price': 1.0001236643976918, 'volume_...","{'slug': 'medium-of-exchange', 'name': 'Medium..."


In [232]:
def get_coin_info(list_of_coins_, api_key):
    df = pd.DataFrame()
    count =0
    for id in list_of_coins_:
        url = 'https://pro-api.coinmarketcap.com/v2/cryptocurrency/quotes/latest'
        parameters = {
            'id': id,
            'limit': 1000
            }
        headers = {
        'Accepts': 'application/json',
        'X-CMC_PRO_API_KEY': api_key
        }

        session = Session()
        session.headers.update(headers)
        response = session.get(url, params=parameters)
        data = json.loads(response.text)
        #navigate json
        #read to DF to see what we have
        cmc_df = pd.DataFrame.from_dict(data['data'], orient='index')
        #need to explode this, then re join 
        #expand nest columns
        expanded_df = pd.DataFrame.from_dict(cmc_df['quote'][0], orient= 'index')
        #showing USD as an index column, going to drop
        count +=1
        #expanded_df.reset_index(drop = True, inplace= True)
        expanded_df.set_index(pd.Index([count]),inplace= True)
               
        trial = cmc_df.drop(['quote','tags'], axis= 1)
        #had to reset index or set it manually to match the expanded DF 
    
        #trial.reset_index(drop = True, inplace= True)
        #trial.reset_index(drop = True, inplace= True)
        trial.set_index(pd.Index([count]),inplace= True)
        
        #had to add key to get them to append properly
        coin_price_df = pd.concat([trial,expanded_df], axis=1, keys=['trial', 'expanded_df']  )
        #coin_price_df.reset_index(drop = True, inplace= True)

        #this saves it as a coreframe

        #going to try to append this to another dataframe
        #df = pd.DataFrame()
        #coin_price_df.set_index(pd.Index([count]),inplace= True)

        df = df.append(coin_price_df)
        #df.set_index(pd.Index([count]),inplace= True)
    #had to drop level inorder to get ride of multilevel index   
    # #https://sparkbyexamples.com/pandas/pandas-multiindex-dataframe-examples/?msclkid=fba45bb6c10e11eca1d376f5b274bab7 
    df = df.droplevel(0, axis =1)
    return df

        


In [233]:
df = get_coin_info(nine_nine_ids_to_try, config.coin_api_key)

KeyError: 'data'

In [225]:
df

,fully_diluted_market_cap,last_updated,market_cap,market_cap_dominance,percent_change_1h,percent_change_24h,percent_change_30d,percent_change_60d,percent_change_7d,percent_change_90d,...,last_updated,max_supply,name,num_market_pairs,platform,self_reported_circulating_supply,self_reported_market_cap,slug,symbol,total_supply
1,8.718836e+11,2022-04-21T01:13:00.000Z,7.895666e+11,41.0776,0.135232,0.510517,0.916163,4.100178,0.743818,1.817720,...,2022-04-21T01:13:00.000Z,21000000,Bitcoin,9340,None,None,NaN,bitcoin,BTC,1.901733e+07
2,3.714983e+11,2022-04-21T01:13:00.000Z,3.714983e+11,19.3274,-0.014169,0.233290,6.090052,13.395458,-1.084463,2.869553,...,2022-04-21T01:13:00.000Z,None,Ethereum,5652,None,None,NaN,ethereum,ETH,1.204620e+08
3,8.563210e+10,2022-04-21T01:13:00.000Z,8.301545e+10,4.3201,-0.000569,-0.011353,-0.026686,-0.027009,-0.003287,0.001861,...,2022-04-21T01:13:00.000Z,None,Tether,32070,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ET...",None,NaN,tether,USDT,8.561395e+10
4,6.944247e+10,2022-04-21T01:13:00.000Z,6.866872e+10,3.5736,0.163165,0.065619,5.851524,6.252955,-0.929666,-3.888622,...,2022-04-21T01:13:00.000Z,165116760,BNB,802,None,None,NaN,bnb,BNB,1.632770e+08
5,4.988339e+10,2022-04-21T01:13:00.000Z,4.988339e+10,2.5952,0.004751,0.019741,0.008022,-0.009534,-0.002756,0.034176,...,2022-04-21T01:13:00.000Z,None,USD Coin,3771,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ET...",None,NaN,usd-coin,USDC,4.989235e+10
6,7.543081e+10,2022-04-21T01:13:00.000Z,3.628617e+10,1.8878,-0.085608,-1.652029,-9.926222,-5.771164,3.526555,4.273639,...,2022-04-21T01:13:00.000Z,100000000000,XRP,712,None,None,NaN,xrp,XRP,9.998959e+10
7,5.439388e+10,2022-04-21T01:13:00.000Z,3.545020e+10,1.8449,0.091645,-1.274469,19.716053,18.313284,0.987867,-15.319193,...,2022-04-21T01:13:00.000Z,None,Solana,295,None,None,NaN,solana,SOL,5.116169e+08
8,6.995114e+10,2022-04-21T01:13:00.000Z,3.353625e+10,1.7453,-0.125876,1.375981,0.063506,89.355788,7.632038,20.346026,...,2022-04-21T01:13:00.000Z,None,Terra,213,None,None,NaN,terra-luna,LUNA,7.360076e+08
9,4.242365e+10,2022-04-21T01:13:00.000Z,3.182014e+10,1.6555,0.282666,-0.040078,3.040943,-4.754947,-3.188489,-25.339226,...,2022-04-21T01:13:00.000Z,45000000000,Cardano,424,None,None,NaN,cardano,ADA,3.427770e+10
10,3.112464e+10,2022-04-21T01:13:00.000Z,2.110712e+10,1.0981,-0.056035,-0.760895,-9.098742,-6.992050,-2.428279,-0.942944,...,2022-04-21T01:13:00.000Z,None,Avalanche,233,None,None,NaN,avalanche,AVAX,3.958913e+08


,price,volume_24h,volume_change_24h,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,last_updated
4,420.658786,2.022702e+09,10.6199,0.63341,0.178291,-0.794199,6.117143,5.97843,-5.230828,6.868389e+10,3.5767,6.945782e+10,2022-04-21T00:47:00.000Z


In [103]:
#https://coinmarketcap.com/api/documentation/v1/#operation/getV2CryptocurrencyQuotesLatest
#now need to reference the coin ids and pull in data from the api accordinly 

config.coin_api_key

url = 'https://pro-api.coinmarketcap.com/v2/cryptocurrency/quotes/latest'
parameters = {
    'id': [1]
    }
headers = {
'Accepts': 'application/json',
'X-CMC_PRO_API_KEY': config.coin_api_key
}

session = Session()
session.headers.update(headers)
response = session.get(url, params=parameters)
data = json.loads(response.text)


In [104]:
#navigate json
data['data']
#read to DF to see what we have
cmc_df = pd.DataFrame.from_dict(data['data'], orient='index')
#need to explode this, then re join 
cmc_df['quote'][0]

{'USD': {'price': 41451.1353306875,
  'volume_24h': 27912601142.293224,
  'volume_change_24h': 11.804,
  'percent_change_1h': 0.09411368,
  'percent_change_24h': -0.11532505,
  'percent_change_7d': 0.78096388,
  'percent_change_30d': 1.10388286,
  'percent_change_60d': 3.42483063,
  'percent_change_90d': 1.74784875,
  'market_cap': 788289960909.4786,
  'market_cap_dominance': 41.0747,
  'fully_diluted_market_cap': 870473841944.44,
  'last_updated': '2022-04-21T00:20:00.000Z'}}

In [97]:
#expand nest columns
expanded_df = pd.DataFrame.from_dict(cmc_df['quote'][0], orient= 'index')
#showing USD as an index column, going to drop
expanded_df.reset_index(drop = True, inplace= True)
#can double check shape
expanded_df.shape
trial = cmc_df.drop(['quote','tags'], axis= 1)
#had to reset index or set it manually to match the expanded DF 
trial.set_index(pd.Index([0]), inplace= True)
#trial.reset_index(drop = True, inplace= True)

coin_price_df = pd.concat([trial,expanded_df], axis=1 )
coin_price_df.set_index(pd.Index([22]), inplace= True)

#going to try to append this to another dataframe
df = pd.DataFrame()

df = df.append(coin_price_df)
df.head()

,id,name,symbol,slug,num_market_pairs,date_added,max_supply,circulating_supply,total_supply,is_active,...,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,last_updated
22,1,Bitcoin,BTC,bitcoin,9340,2013-04-28T00:00:00.000Z,21000000,19017287,19017287,1,...,0.208146,0.288073,0.536021,0.829391,3.857871,0.565335,7.901044e+11,41.1164,8.724794e+11,2022-04-20T22:44:00.000Z


In [52]:
df

""


In [48]:
coin_price_df

,id,name,symbol,slug,num_market_pairs,date_added,max_supply,circulating_supply,total_supply,is_active,...,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,last_updated
0,1,Bitcoin,BTC,bitcoin,9340,2013-04-28T00:00:00.000Z,21000000,19017287,19017287,1,...,0.208146,0.288073,0.536021,0.829391,3.857871,0.565335,7.901044e+11,41.1164,8.724794e+11,2022-04-20T22:44:00.000Z


In [25]:
trial = cmc_df.drop(['quote','tags'], axis= 1)
trial.shape

(1, 16)

In [27]:
coin_price_df

,id,name,symbol,slug,num_market_pairs,date_added,max_supply,circulating_supply,total_supply,is_active,...,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,last_updated
1,1.0,Bitcoin,BTC,bitcoin,9340.0,2013-04-28T00:00:00.000Z,21000000.0,19017287.0,19017287.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.208146,0.288073,0.536021,0.829391,3.857871,0.565335,7.901044e+11,41.1164,8.724794e+11,2022-04-20T22:44:00.000Z


In [26]:
768620549906.7964/ 40426.70598566949

19012693.0

In [14]:
trial.index

Index(['USD'], dtype='object')